# Import + Functions

In [ ]:
# Import Space
import cv2
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import time
import math
import imageio
import glob, os
import datetime
import heapq
import numpy as np
from scipy.signal import find_peaks_cwt
from matplotlib.patches import Ellipse


def smooth(x, window_len=11, window='hanning'):
    """
    Acceptable inputs for window variable:
        'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError(
            "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        )

    s = np.r_[x[window_len - 1:0:-1], x, x[-2:-window_len - 1:-1]]
    
    if window == 'flat':  #moving average
        w = np.ones(window_len, 'd')
    else:
        w = eval('np.' + window + '(window_len)')

    y = np.convolve(w / w.sum(), s, mode='valid')
    return y


# Return vector with zeroes in place of of values outside min/max bounds
def notchFilter(inputVec, minVec, maxVec=sys.maxsize):
    returnVec = np.copy(inputVec)
    for i in range(inputVec.size):
        if inputVec[i] < minVec or inputVec[i] > maxVec:
            returnVec[i] = 0
    return returnVec


# Count number of nonzero intervals in signal
def peakNumber(cleanVector, minZeroes=1):
    numPeaks = 0
    count = 0
    peak = True
    for datapoint in cleanVector:
        if (datapoint > 0 and peak):
            print(datapoint)
            numPeaks += 1
            peak = False
            count = 0
        else:
            if (datapoint == 0):
                count += 1
            if (count >= minZeroes):
                peak = True
    return numPeaks


def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(
        order, normal_cutoff, btype='high', analog=False)
    return b, a

# Attenuate low frequencies, pass high.
def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y


def butter_lowpass(cutOff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutOff / nyq
    b, a = scipy.signal.butter(order, normalCutoff, btype='low', analog=True)
    return b, a

# Attenuate high frequencies, pass low.
def butter_lowpass_filter(data, cutOff, fs, order=4):
    b, a = butter_lowpass(cutOff, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

# Find aTan2 of y,x vector. Returns degrees within 0-360.
def angle(y, x):
    return ((57.2958 * math.atan2(y, x) + 360) % 360)

# Save values for GUI
wSave = 15
mSave = 2
cSave = 2
lSave = 90

# Optical Flow Tracker

In [ ]:
#Instantiate video
file_path = "recordings/p1.avi"  # Change Me!
cap = cv2.VideoCapture(file_path)

# Check that video is valid
ret, old_frame = cap.read()
if not ret:
    raise Exception("No Frame")

    
# Create some random colors
color = np.random.randint(0,255,(100,3))

# GUI
def nothing(x):
    pass

gui = np.zeros((300,512,3), np.uint8)
cv2.namedWindow('gui')
switch = 'OFF-->ON'
cv2.createTrackbar(switch, 'gui',0,1,nothing)
cv2.createTrackbar('Winsize','gui',wSave,101,nothing)
cv2.createTrackbar('maxLevel','gui',mSave,10,nothing)
cv2.createTrackbar('criteria','gui',cSave,100,nothing)
cv2.createTrackbar('liveCutoff','gui',lSave,255,nothing)
cv2.createTrackbar('minDistance','gui',50,200,nothing)
cv2.createTrackbar('blockSize','gui',20,200,nothing)
cv2.createTrackbar('low','gui',100,255,nothing)
cv2.createTrackbar('high','gui',200,255,nothing)
s = 0

# Loop until switch is flipped
while(s<0.9):
    cv2.imshow('gui',gui)
    
    #Check for escape key
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    # Grab current positions of track bars
    winS = cv2.getTrackbarPos('Winsize','gui')
    maxL = cv2.getTrackbarPos('maxLevel','gui')
    critNum = cv2.getTrackbarPos('criteria','gui')
    minD = cv2.getTrackbarPos('minDistance','gui')
    blockS = cv2.getTrackbarPos('blockSize','gui')
    low = cv2.getTrackbarPos('low','gui')
    high = cv2.getTrackbarPos('high','gui')
    liveCutoff = cv2.getTrackbarPos('liveCutoff','gui')
    s = cv2.getTrackbarPos(switch,'gui')

# Save values for next video
wSave = winS
mSave = maxL
cSave = critNum
lSave = liveCutoff

# Very important, always remember to close windows you aren't using
cv2.destroyAllWindows()

# /GUI

# Grab first frame and manual select features 
pos = []
count = 0
display_frame =  np.copy(old_frame)

# Mouse event for picking corners
def on_mouse_click(event, x, y, flags, frame):
    if event == cv2.EVENT_LBUTTONUP:
        global display_frame
        global count
        count += 1
        pos.append([y, x])
        display_frame = cv2.circle(display_frame,(x,y),5,color[count].tolist(),-1)

# Select corners for tracking until escape key pressed
while True:
        cv2.imshow('display_frame', display_frame)
        cv2.setMouseCallback('display_frame', on_mouse_click)
        s = cv2.waitKey(30) & 0xff
        if s == 27:
            time.sleep(1)
            break

# Parameters for ShiTomasi corner detection
feature_params = dict( maxCorners = 100, qualityLevel = 0.3, minDistance = minD,blockSize = blockS )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (winS,winS), maxLevel = maxL, criteria =
                 (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, critNum, 0.03))

# Take first frame and find corners in it
# Also make a mask



#fgmask = None
HSVframe = cv2.cvtColor(old_frame, cv2.COLOR_BGR2HSV)
highHSV = (255, 255, high)
lowHSV = (255, 255, low)

fgmask = cv2.inRange(HSVframe, lowHSV, highHSV)

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

p0 = cv2.goodFeaturesToTrack(old_gray, mask = fgmask, **feature_params)

# TODO: ALLOW MULTIPLE INPUT 

if len(pos) >= 1:
    for i, coords in enumerate(pos):
        if i == 0:
            p0 = np.array([[[float(coords[1]), float(coords[0])]]], np.float32)
        else: 
            p0 = np.vstack([p0, np.array([[[float(coords[1]+5), float(coords[0]+5)]]], np.float32)])

#p0 = [[[421, 133]], [[374, 86]]]
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# Create empty arrays to track points
pointsList = [[np.array([]), np.array([])] for i in range(len(p0))]

while(True):
    ret,frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    th, old_gray = cv2.threshold(old_gray, liveCutoff, 255, cv2.THRESH_TOZERO_INV);
    th, frame_gray = cv2.threshold(frame_gray, liveCutoff, 255, cv2.THRESH_TOZERO_INV);
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    
    if p1 is None:
        break
    # pyrLK flow point selection/unroll
    good_new = p1[st==1]
    good_old = p0[st==1]
    for i in range(len(p1)):
        x = p1[i][0][0]
        y = p1[i][0][1]
        pointsList[i][0] = np.append(pointsList[i][0], x)
        pointsList[i][1] = np.append(pointsList[i][1], y)

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    
    # Original Frame
    img = cv2.add(frame,mask)
    # Black And White 
    #img = cv2.add(cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR),mask)
    
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # update frame/points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)


fig, ax = plt.subplots()
for i in range(len(pointsList)):
    ax.plot(*pointsList[i], label=("Feature " + str(i)))
    
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-small')
plt.axis([0,1280,0,720])
plt.gca().invert_yaxis()
plt.show()

cv2.destroyAllWindows()
cap.release()

# Data Saves

In [ ]:
pointcopy = np.copy(pointsList)

In [ ]:
pointcopy2 = np.copy(pointsList)

In [ ]:
number = "filename"

In [ ]:
np.save(number + "a", pointcopy)
np.save(number + "b", pointcopy2)

In [ ]:
pointcopy = np.load(number+"a.npy")
fig, ax = plt.subplots()
for i in range(len(pointcopy)):
    ax.plot(*pointcopy[i], label=("Feature " + str(i)))
plt.axis([0,1280,0,720])
plt.gca().invert_yaxis()
plt.show()

In [ ]:
pointcopy2 = np.load(number+"b.npy")
fig, ax = plt.subplots()
for i in range(len(pointcopy2)):
    ax.plot(*pointcopy2[i], label=("Feature " + str(i)))
plt.axis([0,1280,0,720])
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Double Arrow Plot

plt.close()
fig, ax = plt.subplots()
cap = cv2.VideoCapture('recordings/10.avi')

ret, frame = cap.read()


x1a, x1b = pointcopy[0][0][i], pointcopy2[0][0][i]
y1a, y1b = pointcopy[0][1][i], pointcopy2[0][1][i]
x2a, x2b = pointcopy[1][0][i], pointcopy2[1][0][i]
y2a, y2b = pointcopy[1][1][i], pointcopy2[1][1][i]

xa, xb = (x1a + x2a) / 2, (x1b + x2b) / 2
ya, yb = (y1a + y2a) / 2, (y1b + y2b) / 2
xDifa, xDifb = -(y2a - ya), -(y2b - yb)
yDifa, yDifb = x2a - xa, x2b - xb

ax = plt.axes()
ax.imshow(frame)
ax.arrow(xa - 1.5*xDifa, ya - 1.5*yDifa, 3*xDifa, 3*yDifa, head_width=10, head_length=10, fc='r', ec='r')
ax.arrow(xb + 1.5*xDifb, yb + 1.5*yDifb, -3*xDifb, -3*yDifb, head_width=10, head_length=10, fc=(0, 0.89, 1), ec=(0, 0.89, 1))

#Updated to Centre of Arrow

xLen = xb - xa
yLen = yb - ya

ax.arrow(xa, ya, xLen, yLen, head_width=1, head_length=1, color=(0, 1, 0.15),  linestyle='dashed')


plt.axis([200,1200,0,720])
plt.gca().invert_yaxis()

plt.show()


In [ ]:
pointcopy = [pointcopy[1], pointcopy[0]] # Use this to flip the angle graph if it looks backwards

In [ ]:
pointcopy2 = [pointcopy2[1], pointcopy2[0]] # Use this to flip the angle graph if it looks backwards

# Direction and Distance Plots

In [ ]:
# Two Arrows
plt.close()
fig, ax = plt.subplots()
angleA = []
angleB = []
angleSub = []
distance = []
for i in range(0, min(len(pointcopy[0][0]), len(pointcopy2[0][0]))-1, 1):
    
    x1a, x1b = pointcopy[0][0][i], pointcopy2[0][0][i]
    y1a, y1b = pointcopy[0][1][i], pointcopy2[0][1][i]
    x2a, x2b = pointcopy[1][0][i], pointcopy2[1][0][i]
    y2a, y2b = pointcopy[1][1][i], pointcopy2[1][1][i]
   
    
    xa, xb = (x1a + x2a) / 2, (x1b + x2b) / 2
    ya, yb = (y1a + y2a) / 2, (y1b + y2b) / 2
    xDifa, xDifb = -(y2a - ya), -(y2b - yb)
    yDifa, yDifb = x2a - xa, x2b - xb
   
    #Updated to Centre of Arrow
    
    xLen = xb - xa
    yLen = yb - ya
    

    angleA.append(angle(yDifa, xDifa))
    angleB.append(angle(yDifb, xDifb))
    tempAngle = abs(angle(yDifa, xDifa) - angle(yDifb, xDifb))
    angleSub.append(tempAngle if tempAngle <= 180 else 360 - tempAngle)
    distance.append(math.sqrt((xLen)**2 + (yLen)**2))
    
winlen = 10
# Plots
size = smooth(np.array(distance), window_len=winlen, window='hanning').size
index = np.linspace(0, size / 30, size)
print("Distance vs Time")
plt.plot(index, smooth(np.array(distance), window_len=winlen, window='hanning'))
plt.show()
print("Angle a vs Time")
plt.plot(index, smooth(np.array(angleA), window_len=winlen, window='hanning'))
plt.show()
print("Angle b vs Time")
plt.plot(index, smooth(np.array(angleB), window_len=winlen, window='hanning'))
plt.show()
print("Angle Difference vs Time")
plt.plot(index, smooth(np.array(angleSub), window_len=winlen, window='hanning'))
plt.show()


In [ ]:
np.save(number+"distance", distance)
np.save(number+"angle", angleSub)
